[View in Colaboratory](https://colab.research.google.com/github/saitota/ProgrammingKnock100/blob/master/Chapter04_Q30_Q39.ipynb)

# :muscle: 前書き
詳細は[第1章](https://qiita.com/saitotak/items/73852ecb7cc4b10b42a5)参照、引き続き言語処理100本ノックの第3章を回答していきます。本章では TBD


# :muscle: 第4章: 形態素解析
> 夏目漱石の小説『吾輩は猫である』の文章（[neko.txt](http://www.cl.ecei.tohoku.ac.jp/nlp100/data/neko.txt)）をMeCabを使って形態素解析し，その結果をneko.txt.mecabというファイルに保存せよ．このファイルを用いて，以下の問に対応するプログラムを実装せよ．
> なお，問題37, 38, 39は[matplotlib](http://matplotlib.org/)もしくは[Gnuplot](http://www.gnuplot.info/)を用いるとよい．

UTF-8,LF の1万行程度の日本語プレーンテキストでした。形態素解析と統計のライブラリを存分に使うことになりそうな感じです。

```text:neko.txt
一

　吾輩は猫である。
名前はまだ無い。

　どこで生れたかとんと見当がつかぬ。
何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。
（以下略）
```

## ⚾ 30. 形態素解析結果の読み込み
> 形態素解析結果（neko.txt.mecab）を読み込むプログラムを実装せよ．ただし，各形態素は表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をキーとするマッピング型に格納し，1文を形態素（マッピング型）のリストとして表現せよ．第4章の残りの問題では，ここで作ったプログラムを活用せよ．


## 下準備として
- ファイルダウンロード
- janomeのインストール

In [1]:
import urllib.request
urllib.request.urlretrieve('http://www.cl.ecei.tohoku.ac.jp/nlp100/data/neko.txt','./neko.txt')


('./neko.txt', <http.client.HTTPMessage at 0x7f50e934e828>)

In [2]:
!pip install janome

    100% |████████████████████████████████| 20.0MB 1.8MB/s 
  Running setup.py bdist_wheel for janome ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - done
  Stored in directory: /content/.cache/pip/wheels/53/60/be/fe884e2d0ebc9fec0988736cf08a2820ab34e3569fc0c5a25a
Successfully built janome


### 解0-1 janomeによる形態素解析と、neko.txt.janome のテキスト作成（おそい）
- MeCab ではなく janome をインストールします、なぜならインストールが簡単だったから。
- `janome.tokenizer` を使って形態素解析します
    - [janome package — janome.tokenizer](http://mocobeta.github.io/janome/api/janome.html#module-janome.tokenizer)
- やたら遅かったので、開始・終了 時間を計測するコードを入れました
- 実測：42sec


In [3]:
from janome.tokenizer import Tokenizer
import janome
import datetime

print('start:' + str(datetime.datetime.now()))

t = Tokenizer()

# WriteのたびにOpen
with open ('./neko.txt','r',encoding='utf-8') as file_temp:
    for line in file_temp:
        # 形態素解析して保存
        for token in t.tokenize(line, stream=True):
            with open ('./neko.txt.janome-01','a',encoding='utf-8') as file_janome:
                    file_janome.write(str(token) + '\n')

print('end:' + str(datetime.datetime.now()))

'''
一	名詞,数,*,*,*,*,一,イチ,イチ
吾輩	名詞,代名詞,一般,*,*,*,吾輩,ワガハイ,ワガハイ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
猫	名詞,一般,*,*,*,*,猫,ネコ,ネコ
で	助動詞,*,*,*,特殊・ダ,連用形,だ,デ,デ
ある	助動詞,*,*,*,五段・ラ行アル,基本形,ある,アル,アル
。	記号,句点,*,*,*,*,。,。,。
名前	名詞,一般,*,*,*,*,名前,ナマエ,ナマエ
（以下略）
'''


start:2018-05-15 05:59:23.543077
end:2018-05-15 06:00:01.161180


'\n一\t名詞,数,*,*,*,*,一,イチ,イチ\n吾輩\t名詞,代名詞,一般,*,*,*,吾輩,ワガハイ,ワガハイ\nは\t助詞,係助詞,*,*,*,*,は,ハ,ワ\n猫\t名詞,一般,*,*,*,*,猫,ネコ,ネコ\nで\t助動詞,*,*,*,特殊・ダ,連用形,だ,デ,デ\nある\t助動詞,*,*,*,五段・ラ行アル,基本形,ある,アル,アル\n。\t記号,句点,*,*,*,*,。,。,。\n名前\t名詞,一般,*,*,*,*,名前,ナマエ,ナマエ\n（以下略）\n'

### 解0-2 高速化
- メモリ枯渇を鑑み 形態素解析→write を繰り返すようにしてみましたが、やたらと時間がかかったため、一括で書き込むようにして高速化を図ります
- 実測：29sec

In [4]:
print('start:' + str(datetime.datetime.now()))

t = Tokenizer()

# Openしてから
with open ('./neko.txt','r',encoding='utf-8') as file_temp , open ('./neko.txt.janome-02','a',encoding='utf-8') as file_janome:
    for line in file_temp:
        # 形態素解析して保存
        for x in t.tokenize(line, stream=True):
            file_janome.write(str(x) + '\n') 
            

print('end:' + str(datetime.datetime.now()))

start:2018-05-15 06:00:01.826460
end:2018-05-15 06:00:27.261072


### 解0-3 さらに高速化
- それでも遅い気がしたので、書き込み部分をリスト内包表記にしました、多少早くなった
- 実測：27sec


In [5]:
print('start:' + str(datetime.datetime.now()))

t = Tokenizer()

# 一部リスト内包表記
with open ('./neko.txt','r',encoding='utf-8') as file_temp , open ('./neko.txt.janome-03','a',encoding='utf-8') as file_janome:
    for line in file_temp:
        # 形態素解析して保存
        [file_janome.write(str(x) + '\n') for x in t.tokenize(line, stream=True)]
            

print('end:' + str(datetime.datetime.now()))

start:2018-05-15 06:00:28.087310
end:2018-05-15 06:00:52.632747


### 解0-4 さらに高速化・・・失敗
- 読みにくいですがぜんぶリスト内包表記にしました
- 実測：88sec
  - 遅くなった何故。。。

In [6]:
print('start:' + str(datetime.datetime.now()))

t = Tokenizer()

# リスト内包表記
with open ('./neko.txt','r',encoding='utf-8') as file_temp , open ('./neko.txt.janome-04','a',encoding='utf-8') as file_janome:
    [
        [
            file_janome.write(str(x) + '\n')
            for x in t.tokenize(y, stream=True)
        ]
        for y in file_temp
    ]

print('end:' + str(datetime.datetime.now()))

start:2018-05-15 06:00:53.349604
end:2018-05-15 06:01:18.101472


### 解1 neko.txt.janome のテキスト作成
-  形態素はタブ区切り、品詞等の分類はカンマ区切りなので成型します

In [7]:
with open ('./neko.txt.janome-04','r',encoding='utf-8') as file_temp:
    text = file_temp.readlines()

lst_janome=[]
for line in text:
    lst_janome.append(
        { 'surface': line.split('\t')[0], # 表層形
          'pos'    : line.split('\t')[1].split(',')[0], # 品詞
          'pos1'   : line.split('\t')[1].split(',')[1], # 品詞細分類1
          'base'   : line.split('\t')[1].split(',')[6]   # 基本形
        })

for i in range(10):
    print(lst_janome[i])
  

{'surface': '一', 'pos': '名詞', 'pos1': '数', 'base': '一'}
{'surface': '吾輩', 'pos': '名詞', 'pos1': '代名詞', 'base': '吾輩'}
{'surface': 'は', 'pos': '助詞', 'pos1': '係助詞', 'base': 'は'}
{'surface': '猫', 'pos': '名詞', 'pos1': '一般', 'base': '猫'}
{'surface': 'で', 'pos': '助動詞', 'pos1': '*', 'base': 'だ'}
{'surface': 'ある', 'pos': '助動詞', 'pos1': '*', 'base': 'ある'}
{'surface': '。', 'pos': '記号', 'pos1': '句点', 'base': '。'}
{'surface': '名前', 'pos': '名詞', 'pos1': '一般', 'base': '名前'}
{'surface': 'は', 'pos': '助詞', 'pos1': '係助詞', 'base': 'は'}
{'surface': 'まだ', 'pos': '副詞', 'pos1': '助詞類接続', 'base': 'まだ'}



## ⚾ 31. 動詞
> 動詞の表層形をすべて抽出せよ．

## 解1 普通にループ

In [8]:
for janome in lst_janome:
    if janome.get('pos') == '動詞':
        print(janome.get('surface'))
    

生れ
つか
し
泣い
し
いる
始め
見
聞く
捕え
煮
食う
思わ
載せ
られ
持ち上げ
られ
し
あっ
落ちつい
見
見
思っ
残っ
いる
さ
れ
し
逢っ
出会わ
し
のみ
なら
し
いる
吹く
せ
弱っ
飲む
知っ
坐っ
おっ
する
し
始め
動く
動く
分ら
廻る
なる
助から
思っ
いる
さり
し
出
し
いる
考え出そ
分ら
付い
見る
い
おっ
見え
隠し
しまっ
違っ
明い
い
られ
這い出し
見る
棄て
られ
這い出す
ある
坐っ
し
考え
見
出
し
泣い
来
くれる
考え付い
やっ
見
来
渡っ
かかる
減っ
来
泣き
出
ある
ある
し
そろ
廻り
始め
し
這っ
行く
出
這入っ
なる
思っ
崩れ
もぐり込ん
破れ
い
し
知れ
云っ
至る
する
なっ
いる
忍び込ん
分ら
なる
減る
降っ
来る
出来
なっ
あるい
行く
考える
這入っ
おっ
見る
し
逢っ
見る
つかん
抛り出し
思っ
ねぶっ
任せ
い
出来
見
這い
上っ
投げ出さ
れ
投げ出さ
れ
這い
上り
這い
上っ
投げ出さ
れ
繰り返し
し
いる
云う
なっ
し
やっ
下り
つまみ出さ
れよ
し
いい
出
来
ぶら下げ
向け
出し
出し
上っ
来
困り
いう
撚り
眺め
おっ
置い
やれ
這入っ
しまっ
聞か
見え
抛り出し
極める
し
合せる
帰る
這入っ
出
来る
思っ
いる
見せ
いる
いう
覗い
見る
し
いる
ある
読みかけ
ある
たらし
いる
帯び
あらわし
いる
食う
食っ
飲む
飲ん
ひろげる
読む
なる
垂らす
繰り返す
考える
ある
生れ
なる
限る
寝
い
勤まる
出来
云わ
せる
来る
鳴らし
いる
住み込ん
行っ
跳ね
付け
られ
し
くれ
さ
れ
至る
つけ
くれ
分る
出来
得る
入れ
くれ
いる
つとめ
読む
乗る
する
乗る
構い
やむを得ん
寝る
し
入っ
もぐり込ん
ねる
なる
入っ
寝る
見出し
割り込む
醒ます
なる
来
来
いっ
泣き
出す
さまし
飛び出し
くる
叩か
れ
し
すれ
する
せ
得
なっ
する
至っ
し
かぶせ
抛り出し
押し込ん
する
しよ
追い廻し
加える
磨い
怒っ
入れ
え
い
する
逢う
言っ
持っ
行っ
棄て
来たそ
流し
話し
する
戦っ
せ
なら
いわ
れ
思う
隣り
解し
い
いっ
し
いる
見


する
っ
し
被っ
いら
笑う
いらっしゃる
尋ねる
こり
固まっ
頂い
いる
あり
撫で
廻す
云っ
おっしゃい
聞い
おり
する
笑う
ありゃ
なっ
思っ
する
付け
いらっしゃる
する
負け
なさる
あり
あっ
割り出し
さ
れ
嘆ぜ
得
立ち至り
なっ
云う
笑い
云う
向う
踏み込ん
見
聞い
始め
上っ
返
食う
なら
図ら
受け
し
し
思い
浮べ
見る
寝転ん
い
られ
なっ
なら
堪え
し
知ら
欠け
し
いる
し
し
いる
せよ
過ぎる
言っ
云う
し
いる
寄り
付ける
云う
過ぎる
し
与える
かろ
し
見る
する
なる
し
乗り込ん
し
やら
読ん
叩きつける
する
し
いる
する
畳み込ん
ある
なっ
云う
する
云え
好み
愛する
する
至る
振り
わす
忍ばし
逢う
する
至る
し
及ぼし
顧み
ある
する
すてる
着い
押す
なる
知れ
申さ
れ
云わ
出掛けよ
起し
飛ん
出
待て
考え
達し
いる
なら
劣ら
れ
忍び込ん
見届け
教え
やる
行か
話せ
話せ
すれ
ある
受け
光ら
なる
やめよ
佇ん
見
思い立っ
やめる
来る
待っ
いる
通り過ぎ
あれ
やる
進む
知る
折り
汚す
生れ
する
出来
する
知る
知ら
告げ
られ
知ら
れ
いる
云う
与
うる
出
来
行か
い
られ
行く
致そ
来
見る
聞い
し
いる
構え
いる
這入っ
眺め
見
しよ
立っ
いる
見
通り抜け
廻る
ある
書い
あっ
劣る
思う
し
いる
這入り
込む
見る
叩き上げ
立ち
焚き
弁じ
いる
かくれる
知ら
云う
知ら
ある
知ら
云え
来
知ら
知っ
てりゃ
恐れる
知れ
知ら
云う
恐れ
してやろ
過ぎる
喰わ
云う
見
思っ
いる
やれ
切れ
提げ
行く
し
いる
行っ
いっ
やる
し
恐れ入る
見せ
聞かし
し
出来る
じらし
やれ
言い付け
おい
なすっ
分っ
いる
引き受ける
云う
示す
冷やかし
来る
通り抜け
這入る
し
歩い
し
踏む
行く
打つ
鼓する
する
行き
行っ
聞き
聞い
出し
立て
帰る
掛け
ある
受け
おり
せ
疑う
ある
云う
する
詰め込ん
ある
知ら
する
踏み
潰す
し
する
気が付い
見る
置か
れ
する
し
祈ら
し
見
違う
見
し
なら
見よ
廻す
廻る
追付こ
思っ
ねじる
とっ


侮る
いる
逆らっ
出来
思わ
与える
云う
ある
かこう
する
見る
ござら
のみ
なら
飛ぶ
違っ
飛ん
しまっ
登ら
云う
する
限ら
かけ
られる
ある
し
逃げる
垂れん
致し
飛ぶ
仕る
云う
及ぼす
しら
出
で
逃げ出す
作る
知ら
吐き
見せ
弄する
入る
なる
かせ
すれ
ある
し
帰る
注する
する
注する
号する
い
重なる
見え
茂っ
いる
なる
すれ
見え
云う
作っ
怪しま
れる
行く
なっ
いる
し
する
来る
立て
飛び出す
いる
飛ぶ
いけ
する
劣ら
飛び出す
する
とどめ
ある
登っ
来
見廻
し
振っ
出直す
しよ
陣取っ
待ち合せ
い
なっ
遊ん
思っ
醒め
落ち
い
登る
取っ
来る
え
しまわ
なら
持っ
来
吐き出す
死ん
いる
じゃらし
引っ掻い
忍ん
行っ
延ばし
縮ま
し
し
いる
抑える
ある
つく
つく
揚げ
振
なる
つく
つく
抑える
つく
つく
し
見せ
もらう
なる
蒙っ
頬張っ
しまう
よる
這入っ
つづけ
いる
ある
やる
述べ
おく
云う
滑る
思う
知れ
取る
登り
登る
登る
し
至る
し
いる
滑ら
懸り
すれ
懸り
懸り
上る
上っ
おい
下がる
下がる
ある
なっ
向け
下り
くる
上っ
くずさ
し
降りる
問う
知っ
てる
降りる
下りる
思う
間違っ
てる
落とし
心得
向い
下りる
思う
する
向い
生え
いる
思う
折れ
いる
かけ
引き寄せる
出来る
押し出す
登っ
する
云え
留まる
おけ
落ちる
落ち
過ぎる
もっ
ゆるめ
なら
降りる
落ちる
降りる
思っ
落ちる
する
降りる
降りる
する
落ちる
なる
落ちる
降りる
落ちる
落ちる
緩め
降り
なら
ある
もっ
落ちる
し
なら
申す
し
立てれ
落ちる
出来る
落ちる
変じ
降りる
なる
し
やっ
見
給え
あっ
立た
滑っ
持ち
答える
出来
なる
降りよ
企て
し
落ちる
なる
出来る
称し
云う
する
もっ
しきら
れ
いる
し
いる
過ぎ
云っ
云う
落ち
する
やり
ある
行く
なる
焼い
立っ
いる
ある
返
やっ
見
やる
なる
なる
なる
返
繰り返し
返
巡り
かけ
飛ん
来
正し
とまっ
する
とまる
ある
思っ
通る
除き
かけ
見
笑っ
いる
眺め
いる
拭い
いる
食っ
来
待つ
与

なる
掛け
さめ
いる
寝
いる
向う
なっ
し
し
切ら
なら
云う
出
こ
なる
なる
ある
限っ
好か
れ
連れ添う
し
おら
云っ
見離さ
れ
持て
気に入る
する
し
好か
れ
つけ
いる
なろ
申し
添える
言いつけ
られ
き
し
する
むい
発せ
あっ
あら
し
なっ
知り
云う
担い
行っ
しまっ
叩き
散らす
する
始め
帯び
する
し
いれ
差し支え
至っ
云わ
得
云う
し
いる
かけ
滑ら
せる
し
し
いる
至っ
背負っ
おら
かる
ある
いる
溜っ
いる
云う
ある
やら
知れ
やっ
なら
なら
やる
かたちづくっ
結びつけ
られ
いる
かかわら
至っ
生れ
せ
られ
挙っ
おら
思う
かかわら
せら
れ
違っ
なっ
参っ
向わ
もっ
有り
つける
立っ
立ち上っ
おり
す
思う
し
い
られ
なっ
知り
する
描い
動か
落ちつい
いる
行か
合う
合わ
し
見
なる
し
見れ
する
きまっ
てる
受取る
出来ん
なっ
這出
し
ある
覗い
見る
違わ
舐め
尽し
洩る
かがやい
いる
炊き
移し
かけ
かきまぜ
ある
沸き上がっ
流れ
だし
こびりつい
貼り
つけ
見える
出来
いる
食わせ
思っ
する
なら
行か
思い切っ
してやろ
考え
定め
甘える
うる
泣い
見
顧みる
つい
泣き
立て
起さ
せる
見
帯び
し
足る
信ずる
顧み
知れ
勤まる
よる
あっ
具え
いる
云わ
せる
ある
開け
くれろ
云っ
開け
くれ
出し
くれ
思う
入れ
くれ
立ち
明かし
待つ
出来る
食っ
蒙っ
見え
かけ
上っ
づけ
ある
し
し
鳴い
見せ
ある
ぬすみ
云う
なる
する
泣き
し
見
劣ら
し
いる
生じ
つい
揚げ
はね
除け
つかみ
出し
敲い
砕け
なっ
這入っ
する
くだけ
押し込ん
傾け
引きかえそ
通り過ぎる
洗っ
てる
し
いる
洗う
云っ
行か
れ
洗え
出来る
引きずり出し
撫で
いる
洗う
云う
あっ
足ら
よる
悟っ
いる
知れ
もっ
任じ
いる
抛
出し
とり
かかる
云う
聞き
し
ぶ
云い
引っ張り
返し
ぶ
有し
いる
知っ
てる
起し
なる
引っ張ら
れる
含ん
垂れ
かかる
する
濡れる
着
いる
聞い
見る
教わっ
来
分ら
濡れる
よし
なさい
洒落れ
云う
間違え
い
思い出し
しまう
や

## 解2. 内容表記と重複排除
- このくらいのループであればリスト内包表記のほうがいいかも
- `set()` による重複排除

In [9]:
set([x.get('surface') for x in lst_janome if x.get('pos') == '動詞'])


{'別れ',
 '待て',
 'ぬける',
 '生じ',
 '異な',
 '預っ',
 'もらい',
 '気づかい',
 '欠け',
 '立ち入っ',
 '惚れ',
 'からかう',
 '擦り切れ',
 '売',
 '思いやら',
 'よむ',
 'しくじっ',
 '出',
 '敲く',
 '代っ',
 '痛み入る',
 '弾け',
 '移し',
 '駆ら',
 'ぶらさがっ',
 '気に入っ',
 '入れ違い',
 '覚まし',
 '払え',
 'やって退け',
 '分ら',
 '逢う',
 '騒い',
 '殺さ',
 '打ちつけ',
 '与っ',
 '隠',
 '暮さ',
 '引っ張っ',
 '挙っ',
 '使お',
 '解く',
 '足りん',
 '出来上がっ',
 'ひ',
 '支え',
 '曝さ',
 '背く',
 '盗ま',
 '取り立て',
 '泊っ',
 '迷っ',
 '這う',
 '試みる',
 '見廻す',
 '突っ伏し',
 'えん',
 '下がっ',
 '打ち抜い',
 'ぶらさげ',
 '乗せ',
 '打ち殺し',
 '考えつい',
 '犯し',
 '蒔き',
 '切り上げ',
 '乗りゃ',
 'あぶり',
 '待ち構え',
 'つかみ合い',
 'でる',
 '溯っ',
 '突き飛ばし',
 '果し',
 '釣る',
 '飼い',
 '力め',
 '起き上がる',
 'もたし',
 '飛ばし',
 'いやがれ',
 '填め',
 '移る',
 '重なっ',
 '取り上げる',
 '見上ぐる',
 'とまり',
 '頬張る',
 '見識張っ',
 '見抜い',
 'ひっくり返っ',
 '甘える',
 'なす',
 '洩り',
 '捕え',
 '亡ぼさ',
 '掘り',
 '解さ',
 '通り抜け',
 '渡る',
 'ねる',
 '飾り付け',
 '間に合っ',
 '祟っ',
 '送れ',
 '凝らし',
 '申し聞け',
 '引き摺り込ま',
 'けれ',
 '挙げ',
 'たとえ',
 '緩め',
 'やむをえ',
 'こびりつく',
 'やっつける',
 '浸す',
 '迎え',
 'おどかさ',
 '儲け',
 '乗ら',
 '叩きつける',
 '寄り',
 '跳ね',
 'なさい',
 'きつける',
 '入れよ',
 '儲かっ


## ⚾ 32. 動詞の原形
> 動詞の原形をすべて抽出せよ．


## 解1. 内包表記
- 原型=基本形（base）のようです

In [10]:
set([x.get('base') for x in lst_janome if x.get('pos') == '動詞'])

{'広げる',
 'ぬける',
 '明く',
 'からかう',
 '届ける',
 'よむ',
 'まじなう',
 'すく',
 '叶う',
 '敲く',
 '痛み入る',
 '言い放つ',
 '逆上せる',
 'まつわる',
 'どける',
 '干る',
 '面す',
 '助かる',
 '逢う',
 'こまる',
 '解く',
 '揉む',
 '背く',
 '切り開く',
 'くずす',
 '掻い込む',
 '這う',
 '試みる',
 '見廻す',
 '汚れる',
 '壊す',
 '剰る',
 '考え付く',
 'でる',
 'さばける',
 '表わす',
 'きかかる',
 '立ち至る',
 '釣る',
 '起き上がる',
 '漕ぐ',
 'のろける',
 '移る',
 'なう',
 'とろける',
 'すくむ',
 '取り上げる',
 'のめる',
 '頬張る',
 '漬ける',
 '倒れる',
 '数え立てる',
 '弱る',
 '後れる',
 '生かす',
 '甘える',
 'なす',
 '見透かす',
 '出合う',
 '渡る',
 'ねる',
 '背負う',
 'こびりつく',
 'やっつける',
 '浸す',
 '絞め殺す',
 '曲げる',
 '叩きつける',
 'きつける',
 '呼ばわる',
 '聞き及ぶ',
 '祟る',
 '拗じる',
 '取り合う',
 '窺う',
 '端折る',
 '仰せる',
 '春めく',
 '別れる',
 'ふてる',
 '畳み込む',
 '沸き上がる',
 'とれる',
 '語る',
 '講じる',
 '言い触らす',
 'ふれる',
 '重なる',
 '相成る',
 '卸す',
 '接す',
 'すくう',
 '見くびる',
 '眺める',
 '撰ぶ',
 '引き受ける',
 '懲らしめる',
 '吹き込む',
 '忘れる',
 '分捕る',
 '生き延びる',
 'せまる',
 '整える',
 '設ける',
 'いたる',
 '添う',
 '持てる',
 '当てる',
 '変じる',
 '事足る',
 '詫びる',
 '引き起す',
 '気に入る',
 '起る',
 '遊ばす',
 '纏める',
 '見て取る',
 '和する',
 '打ち立てる',
 'くぐる',
 '釣り込む',
 '引張る',
 'かおる

## ⚾ 33. サ変名詞
> サ変接続の名詞をすべて抽出せよ．

## 解1. 内包表記
- 品詞細分類1がサ変接続のものを探します

In [11]:
set([x.get('base') for x in lst_janome if x.get('pos1') == 'サ変接続'])

{'学問',
 '逢着',
 '判定',
 '洗濯',
 '礼拝',
 '憎悪',
 '罵詈',
 '不足',
 '長生',
 '伍',
 '挽回',
 '滅亡',
 '演繹',
 '記述',
 '隠居',
 '竣功',
 '震動',
 '意識',
 '会計',
 '占領',
 '予言',
 '供給',
 '誘致',
 '購求',
 '揶揄',
 '慶賀',
 '座禅',
 '訓戒',
 '承知',
 '論及',
 '一段落',
 '落葉',
 '追窮',
 '拷問',
 '反駁',
 '執着',
 '紛失',
 '出版',
 '伝習',
 '背面',
 '退屈',
 '一睡',
 '全快',
 '交換',
 '贔屓',
 '遺伝',
 '脱稿',
 '退校',
 '掃除',
 '類似',
 '入籍',
 '営業',
 '陳列',
 '入学',
 '着服',
 '評論',
 '消耗',
 '貧',
 '対面',
 '授業',
 '履行',
 '附加',
 '結合',
 '醸造',
 '迸出',
 '沈思',
 '交渉',
 '恐怖',
 '出席',
 '思惟',
 '腐爛',
 '遠慮',
 '制定',
 '反比例',
 '昏睡',
 '代言',
 '蠢動',
 '捏造',
 '始末',
 '近道',
 '持続',
 '早死',
 '増長',
 '入水',
 '手品',
 '加味',
 '逗留',
 '嫉妬',
 '長生き',
 '爆発',
 '養成',
 '計算',
 '晩酌',
 '記入',
 '会釈',
 '謹聴',
 '罵倒',
 '成立',
 '競技',
 '服膺',
 '徹',
 '凝結',
 '自殺',
 '首肯',
 '批評',
 '発熱',
 '出馬',
 '了見',
 '独創',
 '窺知',
 '転用',
 '大悟',
 '感染',
 '溷濁',
 '換言',
 '体操',
 '情死',
 '着',
 '迫害',
 '捕捉',
 '推知',
 '蔓延',
 '満足',
 '休心',
 'だかん',
 '徴集',
 '賛助',
 '戦勝',
 '加減',
 '病気',
 '行為',
 '寄寓',
 '増加',
 '断念',
 '矛盾',
 '予知',
 '失礼',
 '自賛',
 '摩擦',
 '冷評',
 '束縛',
 '屈従',
 '動作',

## ⚾ 34. 「AのB」
> 2つの名詞が「の」で連結されている名詞句を抽出せよ．

## 解1. 普通にループ
- 名詞 の 名詞 を探します

In [13]:
for i,janome in enumerate(lst_janome):
    if janome.get('base') == 'の':
        if (lst_janome[i-1].get('pos') == '名詞') and (lst_janome[i+1].get('pos') == '名詞'):
          print (lst_janome[i-1].get('base') + janome.get('base') + lst_janome[i+1].get('base'))

彼の掌
掌の上
書生の顔
はずの顔
顔の真中
穴の中
書生の掌
掌の裏
何の事
肝心の母親
藁の上
笹原の中
池の前
池の上
一樹の蔭
垣根の穴
隣家の三
時の通路
一刻の猶予
家の内
彼の書生
以外の人間
前の書生
おさんの隙
おさんの三
胸の痞
家の主人
主人の方
鼻の下
吾輩の顔
自分の住
吾輩の主人
家のもの
うちのもの
彼の書斎
本の上
皮膚の色
本の上
彼の毎夜
以外のもの
主人の傍
彼の膝
膝の上
経験の上
飯櫃の上
炬燵の上
ここのうち
供の寝床
彼等の中間
供の一
例の神経
性の主人
次の部屋
自分の勝手
吾輩の方
台所の板の間
吾輩の尊敬
向の白
玉のよう
そこの家
家の書生
裏の池
親子の愛
もっともの議論
刺の頭
鰡の臍
彼等のため
軍人の家
代言の主人
教師の家
猫の時節
吾輩の家
家の主人
だらけの英文
胃弱の癖
後架の中
平の宗
月の月給
当分の間
下のよう
今更のよう
主人の述懐
彼の友
金縁の眼鏡
主人の顔
内の想像
訳のもの
以太利の大家
金縁の裏
吾輩の後ろ
彼の友
吾輩の輪廓
顔のあたり
上乗の出来
顔の造作
他の猫
不器量の吾輩
吾輩の主人
斯産の猫
斑入りの皮膚
主人の彩色
一種の色
身内の筋肉
主人の予定
座敷の中
悪口の言いよう
人の気
彼の背中
こっちの便利
自己の力量
人間の不徳
吾輩の家
家の裏
浩然の気
小春の穏
日の二
茶の木の根
西側の杉
垣のそば
他の庭
純粋の黒
彼の皮膚
皮膚の上
毛の間
中の大王
吾輩の倍
嘆賞の念
好奇の心
彼の前
小春の風
垣の上
梧桐の枝
枚の葉
枯菊の茂み
真丸の眼
人間の珍重
双眸の奥
吾輩の矮小
額の上
声の底
吾輩の心臓
ここの教師
教師の家
良家の猫
車屋の黒
車屋の黒
主義の的
彼の名
軽侮の念
左の問答
車屋の方
うちの主人
車屋の猫
どこの国
己の後
教師の方
車屋の黒
相当の気焔
畠の中
黒の比較
彼の鼻
鼻の先
彼の気焔
自分の手柄
墻壁の欠
彼の答
鼠の百
掃除の時
うちの亭主
石灰の袋
椽の下
たちの野郎
溝の中
去年の臭気
鼻の頭
反対の結果
うちの亭主
己の御蔭
無学の黒
背中の毛
黒の子分
以外の御馳走
教師の家
教師のよう
吾輩の主人
日の日記
今日の会
質の人
人の妻君
連中のうち
吾輩の水彩
屋の酒
一廉の水彩
吾輩の水彩
野暮の方
芸者の妻

庚申山の麓
私の宿
例の天の河
哇の方
例のヴァイオリン
秋の灯
胴の丸み
琴線の一部
蝦蟇口の中
肝心のところ
ただの人
学校の生徒
クラスの底
世の中の事
床の間の前
同様の速度
学校の生徒
友達のうち
相当の時間
千秋の思
生の方
探偵のよう
喪家の犬
君の話
家の伝
同情の念
先生の冗談
県庁の前
枯柳の数
病院の横
窓の灯
橋の上
秋の夜長
学校の生徒
町の名
秋の日
甘干しの渋柿
約束の十
善の前
夜寒の頃
目貫の両替
下駄の音
独り言のよう
無言のまま
火鉢の周囲
私の顔
右の手
前の方
私の顔
例の店先
蝦蟇口のなか
店のもの
私の顔
外套の下
善の角
はんの木村
庚申山の裾
僕の所
戸袋のなか
ヴァイオリンの隠れ家
従卒のよう
安宅の関
ヴァイオリンの仲間入り
つづらの中
僕の俳句
組の頭領
督の局
小石川の御寺
鈴木の藤
ビールの徳利
鈴木の味
君の事
君の方
人の味
ビールの徳利
隅の方
人形のよう
機外の機
碁盤の上
子の温泉
人のじじい
東京の呉服
屋の隠居
山の中
中の一軒屋
不便の所
人の前
勘弁のしよう
夢の枕
獅子の洞
煙草の持ち主
室のなか
週間の間
爺さんの御馳走
碁盤の上
君の山羊
迹のよう
山上の白雲
絃の素琴
露地の白
つづらの蓋
つづらの底
例のヴァイオリン
自分の魂
長夜の灯影
時のよう
ランプの傍
つづらの底
草の戸
甘干しの柿
遺憾の至り
草履の所在地
寺の鐘
頭の中
秋の夜長
僕の事
心の中
庚申山の南側
赤松の間
絶の平地
鵜の沼
池のまわり
山のなか
池の近辺
演習のため
一枚岩の上
腹の底
人の心
霊の気
御殿のなか
水晶の御殿
御殿の中
自分の腹の中
水晶の御殿
あすの朝
一枚岩の上
自他の区別
沼の奥
満山の秋
秋の梢
今の音
人の声
鳥の声
猿の声
頭のなか
歓迎の当時
狂乱の態度
総身の毛穴
毛脛のよう
肋骨の下
麓の方
君の話
得意のよう
男子のサンドラ・ベロニ
君子の邦
サンドラ・ベロニの講釈
思のほか
風の歌
森の中
君の庚申山
中の嫦娥
沼の怪
崇高の大差
中止の姿
誰の結婚
金田の令嬢
向うの勝手
両君の写真
ぐるの栄
僕のところ
宝の持ち腐れ
満腹の同情
先生の所
沢山のうち
御国の女
金田の方
闇の中
誰の鉢
家の結婚式
金田の方
私の方
下のよう
議論のよう
不用意の際
人の懐中
不

TBD


## ⚾ 35. 名詞の連接
> 名詞の連接（連続して出現する名詞）を最長一致で抽出せよ．

## ⚾ 36. 単語の出現頻度
> 文章中に出現する単語とその出現頻度を求め，出現頻度の高い順に並べよ．

## ⚾ 37. 頻度上位10語
> 出現頻度が高い10語とその出現頻度をグラフ（例えば棒グラフなど）で表示せよ．

## ⚾ 38. ヒストグラム
> 単語の出現頻度のヒストグラム（横軸に出現頻度，縦軸に出現頻度をとる単語の種類数を棒グラフで表したもの）を描け．

## ⚾ 39. Zipfの法則
> 単語の出現頻度順位を横軸，その出現頻度を縦軸として，両対数グラフをプロットせよ．
